In [35]:
import requests, json, csv

In [36]:
api = {
    'gateway': 'https://manifesto-project.wzb.eu/tools/',
    'key': '?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0'
}

In [37]:
def api_request(api_method, *params):
    parameters = ''
    if params:
        for i in params:
            parameters += ('&' + i)
    request_url = api['gateway'] + api_method + '.json' + api['key'] + parameters
    print(request_url)
    return json.loads(requests.get(request_url).text)

def write_csv(data, name):
    path = 'datasheets/' + name
    with open(path, "w", newline='') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for line in data:
                writer.writerow(line)

In [ ]:
api_request('api_get_core', 'key=MPDS2017b')

In [30]:
    api_request('api_metadata', 'keys[]=51320_196410', 'version=2017-2')

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=51320_196410&version=2017-2


{'items': [{'party_id': 51320,
   'election_date': '196410',
   'language': 'english',
   'source': 'CEMP',
   'has_eu_code': 'FALSE',
   'is_primary_doc': 'TRUE',
   'may_contradict_core_dataset': 'FALSE',
   'manifesto_id': '51320_196410',
   'md5sum_text': '2d63d5748857b13028f4a6f7e9d833bd',
   'url_original': None,
   'md5sum_original': None,
   'annotations': False,
   'handbook': None,
   'is_copy_of': None,
   'title': 'Let’s Go With Labour for the New Britain'}],
 'missing_items': []}

In [38]:
    api_request('api_metadata', 'keys[]=51320_197402', 'version=2017-2')

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=51320_197402&version=2017-2


ConnectionError: HTTPSConnectionPool(host='manifesto-project.wzb.eu', port=443): Max retries exceeded with url: /tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=51320_197402&version=2017-2 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x113232940>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
api_request('api_list_metadata_versions') 

In [34]:
api_request('api_texts_and_annotations', 'keys=51320_1997402', 'version=2017-2')

https://manifesto-project.wzb.eu/tools/api_texts_and_annotations.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys=51320_1997402&version=2017-2


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
api_request('api_get_core_codebook', 'key=MPDS2017b')

https://manifesto-project.wzb.eu/tools/api_get_core_codebook.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&key=MPDS2017b
